source /c/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Mon_env/Scripts/activate

In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Pour afficher joliment les json
from pprint import pprint
# Pour afficher des barres de progression dans une boucle
from tqdm import tqdm
# gestion des temps
import time
# Pour faire des requêtes web (API)
import requests

movie = pd.read_csv('datasets/raw/movie.csv')

#convertir les colonnes (genres, actors, actresses, directors) en liste

movie['genres'] = movie['genres'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
movie['actors'] = movie['actors'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
movie['actresses'] = movie['actresses'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
movie['directors'] = movie['directors'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
movie = movie.drop('vote_count', axis=1)
movie


,tconst,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,spoken_languages,decennie_x,averageRating,numVotes,startYear,runtimeMinutes,genres,actors,actresses,directors
0,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,3.770,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,"['DE', 'FR', 'IT', 'JP', 'PL']",1962-06-22,"['it', 'ja', 'pl', 'fr', 'de']",1960.0,7.2,2515,1962.0,110.0,"[Drama, Romance]","[Jean-Pierre Léaud, Patrick Auffay, François...","[Marie-France Pisier, Rosy Varte, Cristina G...","[Shintarô Ishihara, Marcel Ophüls, Renzo Ros..."
1,tt0266697,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",31.556,/v7TaX8kXMXs5yFFGR41guUDNcnB.jpg,['US'],2003-10-10,"['en', 'ja', 'fr']",2000.0,8.2,1249380,2003.0,111.0,"[Action, Crime, Thriller]","[David Carradine, Michael Madsen, Shin'ichi ...","[Uma Thurman, Daryl Hannah, Lucy Liu, Vivic...",[Quentin Tarantino]
2,tt0116996,en,Mars Attacks!,A fleet of Martian spacecraft surrounds the wo...,21.051,/hll4O5vSAfnZDb6JbnP06GPtz7b.jpg,['US'],1996-12-12,"['en', 'fr']",1990.0,6.4,252014,1996.0,106.0,"[Comedy, Sci-Fi]","[Jack Nicholson, Jack Nicholson, Pierce Bros...","[Sarah Jessica Parker, Annette Bening, Glenn...",NaN
3,tt0112471,en,Before Sunrise,A young man and woman meet on a train in Europ...,22.325,/kf1Jb1c2JAOqjuzA3H4oDM263uB.jpg,"['AT', 'CH', 'US']",1995-01-27,"['en', 'de', 'fr']",1990.0,8.1,358469,1995.0,101.0,"[Comedy, Drama, Romance]","[Ethan Hawke, Hanno Pöschl, Karl Bruckschwai...","[Julie Delpy, Andrea Eckert, Erni Mangold]",[Richard Linklater]
4,tt0381681,en,Before Sunset,"Nine years later, Jesse travels across Europe ...",16.184,/gycdE1ARByGQcK4fYR2mgpU6OO.jpg,['US'],2004-02-10,"['en', 'fr']",2000.0,8.1,301797,2004.0,80.0,"[Drama, Romance]","[Ethan Hawke, Vernon Dobtcheff, Rodolphe Pau...","[Julie Delpy, Louise Lemoine Torrès, Mariane...",[Richard Linklater]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5219,tt0468834,lb,Perl oder Pica,"October 1962, the height of the Cuban missile ...",1.646,/tNCP4nJ66UPN9qMUPvglOzk1kQ3.jpg,['LU'],2006-10-12,"['fr', 'lb']",2000.0,6.1,180,2006.0,90.0,[Drama],"[Ben Hoscheit, André Jung, Thierry van Werve...","[Nicole Max, Anouk Wagener, Myriam Muller, ...",[Pol Cruchten]
5220,tt1873586,fr,Nino (Une adolescence imaginaire de Nino Ferrer),NaN,0.601,NaN,['FR'],2012-04-24,['fr'],2010.0,6.5,21,2011.0,72.0,[Romance],"[David Prat, Benoît Gruel, Benoît Michel, M...","[Sarah Coulaud, Lou de Laâge, Anne Hiribarre...",[Thomas Bardinet]
5221,tt1930420,fr,Noces éphémères,NaN,0.600,NaN,['FR'],2011-10-08,"['fr', 'fa']",2010.0,6.6,34,2011.0,78.0,[Drama],"[Hossein Farzizade, Mohammad Javad Taheri, D...","[Mahnaz Mohammadi, Clothilde Joulin, Masoume...",[Reza Serkanian]
5222,tt2442772,es,Barbazul,"Based on the classic fairytale ""La Barbe Bleue...",0.600,/9ZEjSslzoRkblQrPZTXSYpKMRhC.jpg,['BO'],2012-10-11,"['es', 'fr', 'en']",2010.0,6.1,34,2012.0,98.0,[Horror],"[Jac Avila, Roberto Lopez, Erix Antoine]","[Veronica Paintoux, Mila Joya, Paola Terán, ...",[Amy Hesketh]


# Récupération des films et du casting

In [9]:
import requests
import pandas as pd
import time

API_KEY = "3d79d38c50b154c6468f96037645635e"  # Remplace par ta vraie clé
DISCOVER_URL = "https://api.themoviedb.org/3/discover/movie"
DETAILS_URL = "https://api.themoviedb.org/3/movie"

# Étape 1 : Récupération de tous les films

params = {
    "api_key": API_KEY,
    "language": "fr-FR",
    "sort_by": "popularity.desc",
    "include_adult": "false",
    "include_video": "false",
    "primary_release_date.gte": "2024-01-01",
    "primary_release_date.lte": "2027-12-31",
    "page": 1
}

all_movies = []
total_pages = 1

print(" Récupération des films (Discover)...")

while params["page"] <= total_pages:
    response = requests.get(DISCOVER_URL, params=params)
    
    if response.status_code != 200:
        print(f"Erreur page {params['page']}: {response.status_code}")
        break

    data = response.json()
    all_movies.extend(data["results"])

    total_pages = data["total_pages"]
    print(f"Page {params['page']} sur {total_pages} récupérée.")

    params["page"] += 1
    time.sleep(0.25)

df_movies = pd.DataFrame(all_movies)

print(f"\nTotal films récupérés (Discover): {len(df_movies)}")


# Étape 2 : Récupération des détails pour chaque film 

def get_movie_details(movie_id):
    url = f"{DETAILS_URL}/{movie_id}"
    detail_params = {
        "api_key": API_KEY,
        "language": "fr-FR"
    }
    response = requests.get(url, params=detail_params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erreur pour l'ID {movie_id}: {response.status_code}")
        return None

filtered_movies = []
print("\nRécupération des Movie Details et filtrage par durée...")

for idx, movie_id in enumerate(df_movies["id"]):
    details = get_movie_details(movie_id)
    if details and details.get("runtime") is not None:
        runtime = details["runtime"]
        if 45 <= runtime <= 240:
            filtered_movies.append(details)
    
    if idx % 10 == 0:
        print(f"{idx + 1} films traités...")

    time.sleep(0.25)

df_filtered = pd.DataFrame(filtered_movies)
print(f"\n Films avec durée entre 45 et 240 minutes : {len(df_filtered)}")

# df_filtered => devient movie_tmdb

 Récupération des films (Discover)...
Page 1 sur 3361 récupérée.
Page 2 sur 3361 récupérée.
Page 3 sur 3361 récupérée.
Page 4 sur 3361 récupérée.
Page 5 sur 3361 récupérée.
Page 6 sur 3361 récupérée.
Page 7 sur 3361 récupérée.
Page 8 sur 3361 récupérée.
Page 9 sur 3361 récupérée.
Page 10 sur 3361 récupérée.
Page 11 sur 3361 récupérée.
Page 12 sur 3361 récupérée.
Page 13 sur 3361 récupérée.
Page 14 sur 3361 récupérée.
Page 15 sur 3361 récupérée.
Page 16 sur 3361 récupérée.
Page 17 sur 3361 récupérée.
Page 18 sur 3361 récupérée.
Page 19 sur 3361 récupérée.
Page 20 sur 3361 récupérée.
Page 21 sur 3361 récupérée.
Page 22 sur 3361 récupérée.
Page 23 sur 3361 récupérée.
Page 24 sur 3361 récupérée.
Page 25 sur 3361 récupérée.
Page 26 sur 3361 récupérée.
Page 27 sur 3361 récupérée.
Page 28 sur 3361 récupérée.
Page 29 sur 3361 récupérée.
Page 30 sur 3361 récupérée.
Page 31 sur 3361 récupérée.
Page 32 sur 3361 récupérée.
Page 33 sur 3361 récupérée.
Page 34 sur 3361 récupérée.
Page 35 sur 3361 ré

KeyboardInterrupt: 

In [ ]:
# Export de df_filtered en CSV
movie_tmdb.to_csv('C:/Users/sirnb/OneDrive/Bureau/WildCode/Projets/Projet_2_App/Table_CSV_export/movie_tmdb.csv', index=False)